<a href="https://colab.research.google.com/github/Joaof14/PCM-Cod/blob/main/projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Sistema de Transmissão de Dados


## Pulse Code Modulation e Codificacao de Linha

### Configurando ambiente

In [ ]:
#Importações de bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy import signal
import warnings
warnings.filterwarnings("ignore")  # Ignorar avisos do matplotlib



In [ ]:
# Configurações
n_bits = 8  # Bits por amostra
fs_audio = 44100  # Taxa de amostragem do áudio (Hz)
fs_symbol = 44100  # 1 amostra por símbolo (bit)
SNR_dB = np.arange(0, 20.5, 0.5)  # SNR de 0 a 20 dB em passos de 0.5 dB

### Leitura do Audio e Quantizacao PCM

In [ ]:
# Leitura do arquivo de áudio (substitua pelo seu arquivo .wav)
fs_audio, sinal_analogico = wavfile.read('Better-Call-Saul-Intro-_qcJ3fqWT3a0_.wav')
sinal_analogico = sinal_analogico / np.max(np.abs(sinal_analogico))  # Normalização [-1, 1]

# Quantização PCM (8 bits)
L = 2 ** n_bits
sinal_quantizado = np.round(sinal_analogico * (L/2 - 1)) / (L/2 - 1)



In [ ]:
# Codificação PCM para bits
def pcm_encode(sinal, n_bits):
    return [format(int((x + 1) * (2**n_bits - 1)/2), '0{}b'.format(n_bits)) for x in sinal]

bits_pcm = pcm_encode(sinal_quantizado, n_bits)
bits_flat = [int(bit) for palavra in bits_pcm for bit in palavra]  # Lista plana de bits

In [ ]:
##codificacoes de linha
def codificacao_on_off(bits):
    return [1 if bit == 1 else 0 for bit in bits]

def codificacao_polar(bits):
    return [1 if bit == 1 else -1 for bit in bits]

def codificacao_manchester(bits):
    return [1 if bit == 1 else -1 for bit in bits]  # Simplificado (implementação real requer clock)



In [ ]:
# Aplicar codificação
sinal_on_off = codificacao_on_off(bits_flat)
sinal_polar = codificacao_polar(bits_flat)
sinal_manchester = codificacao_manchester(bits_flat)

### Transmissao em Canal AWGN e Deteccao

In [ ]:
def transmissao_awgn(sinal_codificado, snr_db):
    potencia_sinal = np.mean(np.array(sinal_codificado) ** 2)
    potencia_ruido = potencia_sinal / (10 ** (snr_db / 10))
    ruido = np.random.normal(0, np.sqrt(potencia_ruido), len(sinal_codificado))
    sinal_ruidoso = sinal_codificado + ruido
    return sinal_ruidoso

def detector_limiar(sinal_recebido, limiar=0):
    return [1 if x >= limiar else 0 for x in sinal_recebido]

### Calculo da taxxa de erro de bit (BER) para cada codificacao

In [ ]:
ber_on_off = []
ber_polar = []
ber_manchester = []

for snr in SNR_dB:
    # On-Off
    sinal_rx_on_off = transmissao_awgn(sinal_on_off, snr)
    bits_rx_on_off = detector_limiar(sinal_rx_on_off, limiar=0.5)  # Limiar = 0.5 (On-Off)
    ber_on_off.append(np.sum(np.abs(np.array(bits_flat) - np.array(bits_rx_on_off))) / len(bits_flat))

    # Polar
    sinal_rx_polar = transmissao_awgn(sinal_polar, snr)
    bits_rx_polar = detector_limiar(sinal_rx_polar, limiar=0)  # Limiar = 0 (Polar)
    ber_polar.append(np.sum(np.abs(np.array(bits_flat) - np.array(bits_rx_polar))) / len(bits_flat))

    # Manchester (detecção diferencial)
    sinal_rx_manchester = transmissao_awgn(sinal_manchester, snr)
    bits_rx_manchester = detector_limiar(sinal_rx_manchester, limiar=0)
    ber_manchester.append(np.sum(np.abs(np.array(bits_flat) - np.array(bits_rx_manchester))) / len(bits_flat))

### Grafico de BER x Relacao Sinal Ruido (SNR)

In [ ]:
plt.figure(figsize=(10, 6))
plt.semilogy(SNR_dB, ber_on_off, label='On-Off', marker='o')
plt.semilogy(SNR_dB, ber_polar, label='Polar', marker='s')
plt.semilogy(SNR_dB, ber_manchester, label='Manchester', marker='^')
plt.xlabel('SNR (dB)')
plt.ylabel('BER (Bit Error Rate)')
plt.title('Desempenho das Codificações de Linha em Canal AWGN')
plt.grid(True, which="both", ls="--")
plt.legend()
plt.show()